In [12]:
def is_useful_removal(location, map):
    """ Does making this cell passable create a path or just a dead end? """
    if map.passable[location[0]][location[1]]:
        return False
    neighbours = map.get_passable_neighbours(location)
    if len(neighbours) <= 1:
        return False
    return True

def useful_removals(map):
    useful_removals = filter(lambda cell: is_useful_removal(cell, map), map)
    return useful_removals

In [13]:
def h(map, location):
    """ Heuristic function for A* """
    return abs(location[0] - map.end[0]) + abs(location[1] - map.end[1])

In [14]:
class Map:
    def __init__(self, map):
        # self.map = map
        self.rows = len(map)
        self.cols = len(map[0])
        self.start = (0, 0)
        self.end = (self.rows - 1, self.cols - 1)
        self.passable = [[True if map[row][col] == 0 else False for col in range(self.cols)] for row in range(self.rows)]
        from collections import defaultdict
        self.g_score = defaultdict(lambda: float("inf"))
        self.g_score[self.start] = 0

        self.f_score = defaultdict(lambda: float("inf"))
        self.f_score[self.start] = h(self, self.start)

    def locations(self):
        for row in range(self.rows):
            for col in range(self.cols):
                yield (row, col)

    def modify(self, location, value):
        self.passable[location[0]][location[1]] = value
        cost_of_changed = self.g_score[location]
        self.g_score[location] = float("inf")
        for loc in self.locations():
            if self.g_score[loc] >= cost_of_changed:
                self.g_score[loc] = float("inf")
                self.f_score[loc] = float("inf")

        

    def get_neighbours(self, loc):
        """ Return a list of neighbors for a given location """
        neighbours = []
        if loc[0] > 0:
            neighbours.append((loc[0] - 1, loc[1]))
        if loc[0] < self.cols - 1:
            neighbours.append((loc[0] + 1, loc[1]))
        if loc[1] > 0:
            neighbours.append((loc[0], loc[1] - 1))
        if loc[1] < self.rows - 1:
            neighbours.append((loc[0], loc[1] + 1))
        return neighbours
    def get_passable_neighbours(self, location):
        return filter(lambda loc: self.passable[loc[0]][loc[1]], self.get_neighbours(location))

    def A_star(self):
        open_set = set(self.start)
        came_from = {}

        while open_set:
            current = min(open_set, key=lambda x: self.f_score[x])
            if current == self.end:
                # return reconstruct_path(came_from, current)
                return self.f_score[current]
            open_set.remove(current)
            for neighbor in self.get_passable_neighbours(current):
                tentative_g_score = self.g_score[current] + 1
                if tentative_g_score < self.g_score[neighbor]:
                    came_from[neighbor] = current
                    self.g_score[neighbor] = tentative_g_score
                    self.f_score[neighbor] = self.g_score[neighbor] + h(neighbor)
                    if neighbor not in open_set:
                        open_set.add(neighbor)
        # No path found
        return False

In [15]:
# def reconstruct_path(came_from, current):
#     total_path = [current]
#     while current in came_from.keys():
#         current = came_from[current]
#         total_path.append(current)
#     return total_path

In [16]:
def solution(map):
    m = Map(map)
    current_min_length = float("inf")
    for removed_cell in useful_removals(m):
        m.passable[removed_cell[0]][removed_cell[1]] = False
        current_length = m.A_star()
        if current_length < current_min_length:
            current_min_length = current_length
        m.passable[removed_cell[0]][removed_cell[1]] = True